In [1]:
# Importando os pacotes necessários

import pandas as pd
from IPython.core.display import display, HTML

pd.set_option('display.max_columns', 6)

In [2]:
def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

# Integração de Dados

---

Até agora, nós realizamos a **limpeza** do nosso conjunto de dados, garantindo a remoção de possíveis impurezas. 

Agora, a próxima etapa do *Pré-processamento* é **combinar dados** de diferentes fontes para obter uma estrutura unificada com informações mais significativas. De fato, é bastante comum em aplicações da vida real a necessidade de extrair informação a partir de dados de diferentes fontes. 

Por exemplo, imagine que você precise analisar a popularidade de uma lista de músicas. No entanto, você possui apenas uma tabela separada listando informações básicas de todas as músicas presentes nos *Charts* do Spotify, incluindo o nome e identificador das músicas e de seus artistas. 


Portanto, para obter as informações de popularidade musical, você decide coletar informações dos *Charts*, onde você obtém uma tabela contendo o score da popularidade e a data de lançamento das músicas. Porém, nesta tabela, a única informação de identificação da música é o seu identificador. Consequentemente, apesar de ter as informações que você precisava, seria difícil dizer qual música você está analisando. 



> **O QUE FAZER AGORA?**

Se quisermos fazer uma análise completa sobre a popularidade das músicas coletadas, precisamos reunir os dados das duas tabelas em um único lugar. 

#### Esse processo é denominado **Integração de dados**


## Exemplo

Nesta seção, nós iremos **integrar** informações sobre as músicas do nosso conjunto de dados a partir de diferentes tabelas. 

Através do *pandas*, nós podemos unir múltiplas tabelas utilizando operações de <ins>junção</ins>. 


Para exemplificar, nós dividimos a tabela `Hits` em dois *DataFrames*, `df1` e `df2`. 


### `df1`
- identificador da música: `song_id`

- título da música: `song_name`
- identificador do(s) artista(s): `artist_id`
- nome do(s) artista(s): `artist_name`

### `df2`
- identificador da música: `song_id`

- popularidade da música: `popularity`
- data de lançamento: `release_date`

##### OBSERVAÇÃO! Ambas as tabelas possuem a coluna `song_id` para identificar cada uma das músicas.


In [3]:
# Lendo os dados e criando os dois DataFrames
df1 = pd.read_table('../dataset/spotify_hits_dataset_complete.tsv',
    usecols=['song_id', 'song_name', 'artist_id', 'artist_name'], # seleciona campos específicos
    encoding='utf-8')
df2 = pd.read_table('../dataset/spotify_hits_dataset_complete.tsv',
    usecols=['song_id', 'popularity', 'release_date'], # seleciona campos específicos
    encoding='utf-8')

display_side_by_side([df1.head(3), df2.head(3)], ['df1', 'df2']) # imprime as 3 primeiras linhas

---

Após a criação dos dois *DataFrames*, iremos explorar um recurso essencial oferecido pelo *pandas* para a junção de *DataFrames*: a função `merge()`.


Essa função implementa vários tipos de junções: 

- um-para-um (1:1)
- um-para-muitos (1:N) 
- muitos-para-muitos (N:N)

O tipo de junção realizada depende **essencialmente** da organização dos conjuntos de dados de entrada. 


Nós mostraremos exemplos simples destes três tipos e discutiremos opções detalhadas mais adiante.

### Junções um-para-um (1:1) 

A junção **um-para-um** é talvez o tipo mais simples de fusão, muito semelhante à concatenação de colunas. 

Neste tipo de junção, cada registro na primeira tabela pode ter **somente um** registro correspondente na segunda tabela, e cada registro na segunda tabela pode ter **somente um** registro correspondente na primeira tabela (como mostrado na figura a seguir).


<img src="figure1.png" alt="Junção 1:1" style="width: 500px;"/>

Neste exemplo, nós unimos as tabelas `df1` e `df2` para reunir informações de popularidade das músicas do Spotify usando a função `merge()`. Para isso, nós passamos as tabelas da esquerda e da direita a serem mescladas, através dos campos `left` e `right`. O último parâmetro a ser setado é o `on`, onde iremos passar a **coluna-chave** que está presente nas duas tabelas. 



In [4]:
# Junção um-para-um
df3 = pd.merge(left=df1, right=df2, on='song_id')
df3.head(3)

,song_id,song_name,artist_id,artist_name,popularity,release_date
0,2rRJrJEo19S2J82BDsQ3F7,Falling,['7uaIm6Pw7xplS8Dy06V6pT'],['Trevor Daniel'],77,2020-03-26
1,3BYIzNZ3t9lRQCACXSMLrT,Venetia,['4O15NlyKLIASxsJ0PrXPfz'],['Lil Uzi Vert'],66,2020-03-06
2,1g3J9W88hTG173ySZR6E9S,Tilidin Weg,['1aS5tqEs9ci5P9KD9tZWa6'],['Bonez MC'],13,2020-07-30


O resultado da junção é um único *DataFrame* que combina as informações das duas entradas, baseado nos valores comuns presentes na coluna `song_id` de ambos os *DataFrames* originais.

**Observação:** essa relação **não** é comum porque, geralmente, as informações relacionadas dessa maneira são armazenadas na mesma tabela.

### Junções  um-para-muitos (1:N)

A junção **um-para-muitos** é usada quando uma das duas **colunas-chave** contém mais de uma entrada por registro (como mostrado na figura a seguir).


<img src="figure2.png" alt="Junção 1:N" style="width: 500px;"/>

Para exemplificar, vamos realizar a junção da tabela resultante do exemplo anterior (i.e., *DataFrame* `df3`) com os sucessos do Spotify na tabela `Charts` (i.e., *DataFrame* `df4`). 

Note que uma música pode aparecer em qualquer semana dos *Charts* do Spotify. Portanto, para qualquer música representada na tabela `df3`, pode haver mais de uma ocorrência na tabela `df4`. Neste caso, a relação entre a tabela `df3` e a tabela `df4` é uma relação **um-para-muitos**.

In [5]:
# Lendo os dados
df4 = pd.read_table('../dataset/spotify_charts_complete.tsv',
    usecols=['chart_week', 'song_id', 'position'], # seleciona campos específicos
    encoding='utf-8')
df4 = df4.sort_values(by=['song_id']) # ordena o DataFrame pelo id da música

display_side_by_side([df3.head(3), df4.head(3)], ['df3', 'df4']) # imprime as 3 primeiras linhas

In [6]:
# Junção um-para-muitos de dois DataFrames
df5 = pd.merge(left=df3, right=df4, on='song_id') 
df5.head(3)

,song_id,song_name,artist_id,...,release_date,chart_week,position
0,2rRJrJEo19S2J82BDsQ3F7,Falling,['7uaIm6Pw7xplS8Dy06V6pT'],...,2020-03-26,2020-10-29,81
1,2rRJrJEo19S2J82BDsQ3F7,Falling,['7uaIm6Pw7xplS8Dy06V6pT'],...,2020-03-26,2020-05-07,20
2,2rRJrJEo19S2J82BDsQ3F7,Falling,['7uaIm6Pw7xplS8Dy06V6pT'],...,2020-03-26,2020-09-17,71


O resultado da junção é um único *DataFrame* que combina as informações das duas entradas; porém, ao contrário do exemplo anterior, as informações originais (i.e., do *DataFrame* `df3`) se repetem conforme exigido pelas entradas do *DataFrame* `df4`.

### Junções  muitos-para-muitos (N:N)

Um relacionamento **muitos-para-muitos** ocorre quando vários registros em uma tabela são associados a vários registros em outra tabela (como mostrado na figura a seguir). 

<img src="figure3.png" alt="Junção N:N" style="width: 500px;"/>

Para exemplificar, considere a tabela resultante anterior (i.e., *DataFrame* `df5`)

um relacionamento muitos para muitos existe entre clientes e produtos: clientes podem comprar vários produtos e produtos podem ser comprados por muitos clientes.

Se a **coluna-chave** das duas tabelas incluir entradas duplicadas, o resultado é uma fusão **muitos-para-muitos**. 



Neste exemplo, para criar um relacionamento **muitos-para-muitos** entre a tabela resultante anterior (i.e., *DataFrame* `df5`) e outra contendo o número total de *streams* que cada música atingiu nas semanas dos charts (i.e., *DataFrame* `df6`), a **coluna-chave** do *DataFrame* resultante é a combinação das **colunas-chave** de cada tabela. 

Agora, as duas tabelas terão duas **colunas-chave**: `song_id` e `chart_week`.


In [7]:
# Lendo os dados
df6 = pd.read_table('../dataset/spotify_charts_complete.tsv',
    usecols=['chart_week', 'song_id', 'streams'], # seleciona campos específicos
    encoding='utf-8')

display_side_by_side([df5.head(3), df6.head(3)], ['df5', 'df6']) # imprime as 3 primeiras linhas

In [8]:
# Junção muitos-para-muitos de dois DataFrames
df7 = pd.merge(left=df5, right=df6, on=['song_id', 'chart_week'])
df7.head(3)

,song_id,song_name,artist_id,...,chart_week,position,streams
0,2rRJrJEo19S2J82BDsQ3F7,Falling,['7uaIm6Pw7xplS8Dy06V6pT'],...,2020-10-29,81,7517622
1,2rRJrJEo19S2J82BDsQ3F7,Falling,['7uaIm6Pw7xplS8Dy06V6pT'],...,2020-05-07,20,16751384
2,2rRJrJEo19S2J82BDsQ3F7,Falling,['7uaIm6Pw7xplS8Dy06V6pT'],...,2020-09-17,71,7944403


Após a junção **muitos-para-muitos**, a **coluna-chave** do *DataFrame* resultante é expressa pela combinação dos valores de `song_id` e `chart_week` das tabelas `df5` e `df6`.


## OPÇÕES DETALHADAS

### Conclusão

Este notebook apresentou como fazer a integração de dados de diferentes fontes.

O próximo notebook ([3.3.Transformacao.ipynb](3.3.Transformacao.ipynb)) apresenta como transformar dados de diferentes formatos em um formato suportado pelo processo de pesquisa.